In [66]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
import numpy as np
import requests
import pymysql
import re
import importlib
from bs4 import BeautifulSoup
import time
# from bs4 import 

In [ ]:
base_url = 'https://pokemongo.inven.co.kr/dataninfo/pokemon/'
base_res = requests.get(base_url)
base_soup = BeautifulSoup(base_res.text, 'html.parser')
base_soup

In [102]:
from traceback import print_tb


detail_url = ['https://pokemongo.inven.co.kr' + b['href'] for b in base_soup.select('#pokemongoList > ul > li> a')]
iter_num = 0
poke_info_dic = {}
for d_url in detail_url:
    detail_res = requests.get(d_url)
    detail_soup = BeautifulSoup(detail_res.text, 'html.parser')
    
    title_soup = detail_soup.select_one('#pokemongoDb .pokemongoDbCommonTitle')
    poke_number = title_soup.text.split('.')[0].strip()
    poke_name = title_soup.text.split('.')[1].strip()
    
    poke_type1 = detail_soup.select_one('.articleContent .pokemontype li.poekmontype').text.split()[0].strip()
    try:
        poke_type2 = detail_soup.select_one('.articleContent .pokemontype li.poekmontype').text.split()[1].strip()
        is_single_type = 'n'
    except:
        poke_type2 = ''
        is_single_type = 'y'
    
    poke_stat = {}
    
    poke_stat['진화 비용']= detail_soup.select_one('.articleContent .pokemontype li.last').text.strip()
    poke_stat['최대 CP'] = int(detail_soup.select_one('#articleView > div.articleContent > div.pokemontype > ul > li:nth-child(6)').text)
    poke_stat['공격력'] = int(detail_soup.select_one('#articleView > div.articleContent > div.pokemontype > ul > li:nth-child(8)').text)
    poke_stat['방어력'] = int(detail_soup.select_one('#articleView > div.articleContent > div.pokemontype > ul > li:nth-child(10)').text)
    poke_stat['체력'] = int(detail_soup.select_one('#articleView > div.articleContent > div.pokemontype > ul > li:nth-child(12)').text)
    poke_stat['포획 확률'] = detail_soup.select_one('#articleView > div.articleContent > div.pokemontype > ul > li:nth-child(14)').text.strip()
    
    type_match_soups = detail_soup.select('#articleView > div.articleContent > div.wrapContent > div.typeData > table > thead > tr')
    type_match_table = {}
    
    for tms in type_match_soups:
        for th in tms.findAll('th'):
            td = th.findNext()
            if td.text.strip() != '-':
                type_match_table[th.text] = float(td.text)
            else:
                type_match_table[th.text] = 1.0

    evol_soup = detail_soup.select_one('#articleView > div.articleContent > div.groupList > ul')
    n = 1
    evol_step = 1
    evol_poke_step_list = []
    evol_poke_name_list = []
    for li in evol_soup.findAll('li'):
        n += 1
        if 'arrow' in li.get_attribute_list('class'):
            evol_step += 1
        else:
            evol_poke_step_list.append(evol_step)
            evol_poke_name_list.append(li.select_one('span > a').text.split('.')[1].strip())
    
    for k, v in enumerate(evol_poke_name_list):
        if poke_name == v:
            break
    poke_evol_step = evol_poke_step_list[k]
    if evol_poke_step_list[-1] == poke_evol_step:
        is_final_evol = 'y'
    else:
        is_final_evol = 'n'
    
    #poke_info_dic 만들기
    poke_info_dic[poke_name] = {
        '이름': poke_name,
        '번호' : poke_number,
        '타입1' : poke_type1,
        '타입2' : poke_type2,
        '단일타입' : is_single_type,
        '스탯' : poke_stat,
        '상성' : type_match_table,
        '진화' : list(zip(evol_poke_step_list, evol_poke_name_list)),
        '단계' : poke_evol_step,
        '최종진화' : is_final_evol
    }
    
    iter_num += 1
    if iter_num > 2:
        break
    time.sleep(1)

In [104]:
print(len(poke_info_dic))
print(poke_info_dic.keys())
poke_info_dic['이상해꽃']

3
dict_keys(['이상해씨', '이상해풀', '이상해꽃'])


{'이름': '이상해꽃',
 '번호': '3',
 '타입1': '풀',
 '타입2': '독',
 '단일타입': 'n',
 '스탯': {'진화 비용': '0',
  '최대 CP': 2720,
  '공격력': 198,
  '방어력': 189,
  '체력': 190,
  '포획 확률': '5%'},
 '상성': {'노말': 1.0,
  '격투': 0.625,
  '비행': 1.6,
  '독': 1.0,
  '땅': 1.0,
  '바위': 1.0,
  '벌레': 1.0,
  '고스트': 1.0,
  '강철': 1.0,
  '불꽃': 1.6,
  '물': 0.625,
  '풀': 0.391,
  '전기': 0.625,
  '에스퍼': 1.6,
  '얼음': 1.6,
  '드래곤': 1.0,
  '악': 1.0,
  '페어리': 0.625},
 '진화': [(1, '이상해씨'), (2, '이상해풀'), (3, '이상해꽃')],
 '단계': 3,
 '최종진화': 'y'}

In [ ]:
import pymysql
import configparser
import os
from pathlib import Path

conf_path = (os.path.dirname(Path.cwd())) + f'\ignore\config.ini'

conf = configparser.ConfigParser()
conf.read(conf_path)

poke_db_conf = dict(conf['POKEMON_DB_CONFIG'])
poke_db_conf['port'] = int(poke_db_conf['port'])
poke_db_conf

conx = pymysql.connect(**poke_db_conf)
cur = conx.cursor()

# if conx.ping():
#     print("Connected")
# else:
#     print("Failed to connect")

{'이름': '이상해씨',
 '번호': '1',
 '타입1': '풀',
 '타입2': '독',
 '단일타입': 'n',
 '스탯': {'진화 비용': '25',
  '최대 CP': 1115,
  '공격력': 118,
  '방어력': 111,
  '체력': 128,
  '포획 확률': '20%'},
 '상성': {'노말': 1.0,
  '격투': 0.625,
  '비행': 1.6,
  '독': 1.0,
  '땅': 1.0,
  '바위': 1.0,
  '벌레': 1.0,
  '고스트': 1.0,
  '강철': 1.0,
  '불꽃': 1.6,
  '물': 0.625,
  '풀': 0.391,
  '전기': 0.625,
  '에스퍼': 1.6,
  '얼음': 1.6,
  '드래곤': 1.0,
  '악': 1.0,
  '페어리': 0.625},
 '진화': [(1, '이상해씨'), (2, '이상해풀'), (3, '이상해꽃')],
 '단계': 1,
 '최종진화': 'n',
 '진화 비용': '25',
 '최대 CP': 1115,
 '공격력': 118,
 '방어력': 111,
 '체력': 128,
 '포획 확률': '20%',
 '노말': 1.0,
 '격투': 0.625,
 '비행': 1.6,
 '독': 1.0,
 '땅': 1.0,
 '바위': 1.0,
 '벌레': 1.0,
 '고스트': 1.0,
 '강철': 1.0,
 '불꽃': 1.6,
 '물': 0.625,
 '풀': 0.391,
 '전기': 0.625,
 '에스퍼': 1.6,
 '얼음': 1.6,
 '드래곤': 1.0,
 '악': 1.0,
 '페어리': 0.625}

In [131]:
conx.commit()